In [1]:
import pandas as pd

In [2]:
## Set-up
import requests
import json

# Insert activated key
key = '1b5678a1a374e17f9c32c8981fbe81d8f437edc7'

In [36]:
def get_pop_estimate(year, geography, specific_area=None):
    '''
    Function to pull Census Bureau population estimates for non-census years.
    
    Parameters
    ----------
    year : int. The year for which you want to get population estimates.
    geography : str. The geographic granularity of the estimate. Options:
        'us' : returns national population estimate.
        'state' : returns state population estimates.
        'county' : returns county estimates.
    specific_area : str. Either 2-digit state FIPS code or 5-digit county FIPS code. If None, will return all areas within the geography.
    '''
    if year%10 == 0:
        print(year,'is a Census year. Use CensusBureau.ipynb instead for more accurate population estimates.')
        return
    
    ## Specify URL
    if specific_area == None:
        specific_area = ['*', '*']
    elif len(specific_area) == 5:
        specific_area = [specific_area[2:], specific_area[:2]]
    elif len(specific_area) == 2:
        specific_area = ['*', specific_area]
    else:
        raise ValueError('specific_area should be 2-digit state FIPS code, 5-digit county FIPS code, or None.')
        
    if geography == 'county':
        geography = f'county:{specific_area[0]}&in=state:{specific_area[1]}'
    elif geography == 'state':
        geography = f'state:{specific_area[1]}'
    
    url = f'http://api.census.gov/data/{year}/pep/population?get=POP,GEONAME&for={geography}&key={key}'
    
    ## Store response
    data = requests.get(url)

    ## Check response
    if data.status_code == 200:
        print('Status code:', data.status_code, '- OK')
    else:
        print('Status code:', data.status_code, '- ERROR')
        
    ## Store API response in a variable
    available_data = data.json()

    ## Create dataframe
    pop_est = pd.DataFrame(available_data[1:], columns = available_data[0])
    pop_est['POP'] = pd.to_numeric(pop_est['POP']) 
    
    ## Return dataframe
    return pop_est